In [25]:
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline

from astropy import table
from astropy.table import Table
from astropy.io import ascii
from tabulate import tabulate

SAGA_DIR = os.environ['SAGA_DIR']


In [26]:
# READ SAGA NAMES DOCUMENT
key = '1GJYuhqfKeuJr-IyyGF_NDLb_ezL6zBiX2aeZFHHPr_s'
gid =0
url = 'https://docs.google.com/spreadsheets/d/{0}/export?format=csv&gid={1}'.format(key, gid)
googlenames = Table.read(url, format='ascii.csv')
m=googlenames['Flag'] == 0
saganames = googlenames[m]

In [27]:
# RUN LOOP ONCE TO SORT ON COMPLETENESS
cmpl = []
for host in saganames:
    nsaid = host['NSA']
    basefile  = os.path.join(SAGA_DIR, 'base_catalogs', 'base_sql_nsa{0}.fits.gz'.format(nsaid))
    base = Table.read(basefile)    
    
    b_rmv  = base['REMOVE'] == -1
    b_rvir = base['RHOST_KPC'] <= 300
    b_fib  = base['FIBERMAG_R'] <= 23
    b_gal  = base['PHOT_SG'] == 'GALAXY'
    b_r21  = base['r'] - base['EXTINCTION_R'] <= 20.75
    
    base = base[b_rmv & b_rvir & b_gal & b_fib & b_r21]

    # CALCULATE UGRI QUANTITIES
    umag = base['u'] - base['EXTINCTION_U']
    gmag = base['g'] - base['EXTINCTION_G']
    rmag = base['r'] - base['EXTINCTION_R']
    imag = base['i'] - base['EXTINCTION_I']

    ug = umag - gmag
    gr = gmag - rmag
    ri = rmag - imag

    grerr = np.sqrt(base['g_err']**2 + base['r_err']**2)
    rierr = np.sqrt(base['r_err']**2 + base['i_err']**2)

    cgmr = gr - 2.*grerr
    crmi = ri - 2.*rierr

    msk1 = cgmr < 0.85
    msk2 = crmi < 0.55
    msk3 = base['ZQUALITY'] >= 3

    # GRI COMPLETENESS
    ngri      = np.sum(msk1&msk2)
    ngri_spec = np.sum(msk1&msk2&msk3)
    p = 100.*ngri_spec/ngri
    cmpl.append([p,nsaid])    # WE WILL SORT ON THIS QUANITY


In [28]:
sorted_hosts = sorted(cmpl,reverse=True)

In [30]:
# from yao, based on median 
mvir = ascii.read('/Users/marlageha/Projects/SAGA/host_vir.dat')
nsa = Table.read('/Users/marlageha/Projects/SAGA/cats/nsa_v0_1_3.fits')

In [37]:
# RUN LOOP AGAIN TO GENERATE TABLE
data = []
headers = ['SAGA','NGC','NSAID','RA','Dec','Dist','Mr','MK','Mstar','Mvir','Ntot','GRI COMPL']

for h in sorted_hosts:
    nsaid = h[1]
    basefile  = os.path.join(SAGA_DIR, 'base_catalogs', 'base_sql_nsa{0}.fits.gz'.format(nsaid))
    base = Table.read(basefile)    
    row = []

    msk = np.where(saganames['NSA']==nsaid)[0][0]
    saga = saganames['SAGA'][msk]    
    ngc = saganames['NGC'][msk]
    
    row.append(saga)
    row.append('NGC '+ngc)
    row.append(nsaid)

    
    row.append('{:09.5f}'.format(np.float(saganames['RA'][msk])))
    row.append('{:09.5f}'.format(np.float(saganames['Dec'][msk])))
    row.append('{:06.1f}'.format(base['HOST_DIST'][0]))
 

    row.append('{:06.1f}'.format(base['HOST_MR'][0]))
    row.append('{0:.1f}'.format(base['HOST_MK'][0]))
    
    # STELLAR AND VIRIAL MASS
    m=nsaid == nsa['NSAID']
    mstar = np.log10(nsa['MASS'][m]/0.7**2)
    row.append('{:06.2f}'.format(np.float(mstar)))
    m=nsaid == mvir['NSAID']
    vir = np.log10(mvir['MVIR'][m])
    row.append('{:06.2f}'.format(np.float(vir)))




    # BASIC CUTS ON BASE CATALOG
    b_rmv  = base['REMOVE'] == -1
    b_rvir = base['RHOST_KPC'] <= 300
    b_fib  = base['FIBERMAG_R'] <= 23
    b_gal  = base['PHOT_SG'] == 'GALAXY'
    b_r21  = base['r'] - base['EXTINCTION_R'] <= 20.75
    
    base = base[b_rmv & b_rvir & b_gal & b_fib]
    b_r21  = base['r'] - base['EXTINCTION_R'] <= 20.75


    # CALCULATE NUMBER OF SATELLITES
    nsat = np.sum(base['SATS'] == 1)
    ncsat = np.sum((base['SATS'] == 1) & (base['r'] - base['EXTINCTION_R'] <= 20.75))

    sats = '{}'.format(nsat)
    row.append(sats)
    
    
    # NOW CALCULATE COMPLETENESSES
    base=base[b_r21]
    
    row.append(np.sum(b_r21))
    
    # CALCULATE UGRI QUANTITIES
    umag = base['u'] - base['EXTINCTION_U']
    gmag = base['g'] - base['EXTINCTION_G']
    rmag = base['r'] - base['EXTINCTION_R']
    imag = base['i'] - base['EXTINCTION_I']

    ug = umag - gmag
    gr = gmag - rmag
    ri = rmag - imag

    grerr = np.sqrt(base['g_err']**2 + base['r_err']**2)
    rierr = np.sqrt(base['r_err']**2 + base['i_err']**2)
    ugerr = np.sqrt(base['u_err']**2 + base['g_err']**2)

    cgmr = gr - 2.*grerr
    crmi = ri - 2.*rierr

    msk1 = cgmr < 0.85
    msk2 = crmi < 0.55
    msk3 = base['ZQUALITY'] >= 3

    # GRI COMPLETENESS
    ngri      = np.sum(msk1&msk2)
    ngri_spec = np.sum(msk1&msk2&msk3)
    p = 100.*ngri_spec/ngri
    gri_compl = '{:02.0f}% ({}/{})'.format(p,ngri_spec,ngri)
    row.append(gri_compl)

     
    # UGRI COMPLETENESS
    #msk4 = ug < 1.5*gr#-0.25
    msk4 = (ug+2*ugerr) > (1.5*(gr - 2.*grerr))
    nugri      = np.sum(msk1&msk2&msk4)
    nugri_spec = np.sum(msk1&msk2&msk3&msk4)
    p = 100.*nugri_spec/nugri
    ugri_compl = '{:02.0f}% ({}/{})'.format(p,nugri_spec,nugri)
#    row.append(ugri_compl)
    
    data.append(row)


  MASS 
-------
10.5198
  MASS 
-------
10.5711
  MASS 
-------
10.5306
  MASS 
-------
10.7024
  MASS 
-------
10.5695
  MASS 
-------
10.5632
  MASS 
-------
10.6356
 MASS 
------
10.709
  MASS 
-------
10.3138
 MASS 
------
10.195
  MASS 
-------
10.6063
  MASS 
-------
10.6228
 MASS 
------
10.415
  MASS 
-------
10.6075
  MASS 
-------
10.4351
  MASS 
-------
10.1815


In [38]:
print tabulate(data,headers, tablefmt="rst")

===========  =============  ======  =========  ========  =====  ======  =====  =====  =======  ======  ======  ===============
             SAGA              NGC      NSAID        RA    Dec    Dist     Mr     MK    Mstar    Mvir    Ntot  GRI COMPL
===========  =============  ======  =========  ========  =====  ======  =====  =====  =======  ======  ======  ===============
Gilgamesh    NGC 5962       166313  234.132    16.6078    28     -21.2  -23.7  10.52    12.13       2    2995  98% (1271/1300)
Odyssey      NGC 6181       147100  248.087    19.8264    34.3   -21.3  -24    10.57    12.27       9    1850  97% (819/845)
Dune         NGC 5750       165536  221.546    -0.22294   25.4   -20.9  -23.6  10.53    12.08       1    3557  97% (1433/1480)
AnaK         NGC 7716        61945  354.131     0.29728   34.8   -21.4  -23.4  10.7     12.01       3    2356  94% (917/979)
Narnia       NGC 1015       132339   39.5482   -1.31876   37.2   -21.1  -23.5  10.57    12.05       2    1976  92% (778/8

In [39]:
print tabulate(data, headers, tablefmt="latex")


\begin{tabular}{llrrrrrrrrrrl}
\hline
             & SAGA          &    NGC &     NSAID &       RA &   Dec &   Dist &    Mr &    MK &   Mstar &   Mvir &   Ntot & GRI COMPL       \\
\hline
 Gilgamesh   & NGC 5962      & 166313 & 234.132   & 16.6078  &  28   &  -21.2 & -23.7 & 10.52 &   12.13 &      2 &   2995 & 98\% (1271/1300) \\
 Odyssey     & NGC 6181      & 147100 & 248.087   & 19.8264  &  34.3 &  -21.3 & -24   & 10.57 &   12.27 &      9 &   1850 & 97\% (819/845)   \\
 Dune        & NGC 5750      & 165536 & 221.546   & -0.22294 &  25.4 &  -20.9 & -23.6 & 10.53 &   12.08 &      1 &   3557 & 97\% (1433/1480) \\
 AnaK        & NGC 7716      &  61945 & 354.131   &  0.29728 &  34.8 &  -21.4 & -23.4 & 10.7  &   12.01 &      3 &   2356 & 94\% (917/979)   \\
 Narnia      & NGC 1015      & 132339 &  39.5482  & -1.31876 &  37.2 &  -21.1 & -23.5 & 10.57 &   12.05 &      2 &   1976 & 92\% (778/849)   \\
 OBrother    & NGC PGC068743 & 149781 & 335.913   & -3.43167 &  39.2 &  -21   & -23.8 & 10.5

In [ ]:
# RUN LOOP AGAIN TO GENERATE TABLE
data = []
headers = ['SAGA','Dist','Ns/Nc','MK','Mr','Ntot','GRI COMPL', 'UGRI COMPL']

for h in sorted_hosts:
    nsaid = h[1]
    basefile  = os.path.join(SAGA_DIR, 'base_catalogs', 'base_sql_nsa{0}.fits.gz'.format(nsaid))
    base = Table.read(basefile)    
    row = []

    msk = np.where(saganames['NSA']==nsaid)[0][0]
    
    ngc = saganames['NGC'][msk]
    saga = saganames['SAGA'][msk]
    #row.append(ngc)
    row.append(saga)
#   row.append(nsaid)

    
#    row.append('{:09.5f}'.format(np.float(saganames['RA'][msk])))
#    row.append('{:09.5f}'.format(np.float(saganames['Dec'][msk])))
    row.append('{:06.1f}'.format(base['HOST_DIST'][0]))
 
    

    # BASIC CUTS ON BASE CATALOG
    b_rmv  = base['REMOVE'] == -1
    b_rvir = base['RHOST_KPC'] <= 300
    b_fib  = base['FIBERMAG_R'] <= 23
    b_gal  = base['PHOT_SG'] == 'GALAXY'
    b_r21  = base['r'] - base['EXTINCTION_R'] <= 20.75
    
    base = base[b_rmv & b_rvir & b_gal & b_fib]
    b_r21  = base['r'] - base['EXTINCTION_R'] <= 20.75


    # CALCULATE NUMBER OF SATELLITES
    nsat = np.sum(base['SATS'] == 1)
    ncsat = np.sum((base['SATS'] == 1) & (base['r'] - base['EXTINCTION_R'] <= 20.75))

    sats = '{}/{}'.format(nsat,ncsat)
    row.append(sats)
    
    row.append('{0:.1f}'.format(base['HOST_MK'][0]))
    row.append('{:06.1f}'.format(base['HOST_MR'][0]))
    
    
    # NOW CALCULATE COMPLETENESSES
    base=base[b_r21]
    
    row.append(np.sum(b_r21))
    
    # CALCULATE UGRI QUANTITIES
    umag = base['u'] - base['EXTINCTION_U']
    gmag = base['g'] - base['EXTINCTION_G']
    rmag = base['r'] - base['EXTINCTION_R']
    imag = base['i'] - base['EXTINCTION_I']

    ug = umag - gmag
    gr = gmag - rmag
    ri = rmag - imag

    grerr = np.sqrt(base['g_err']**2 + base['r_err']**2)
    rierr = np.sqrt(base['r_err']**2 + base['i_err']**2)

    cgmr = gr - 2.*grerr
    crmi = ri - 2.*rierr

    msk1 = cgmr < 0.85
    msk2 = crmi < 0.55
    msk3 = base['ZQUALITY'] >= 3

    # GRI COMPLETENESS
    ngri      = np.sum(msk1&msk2)
    ngri_spec = np.sum(msk1&msk2&msk3)
    p = 100.*ngri_spec/ngri
    gri_compl = '{:02.0f}% ({}/{})'.format(p,ngri_spec,ngri)
    row.append(gri_compl)

     
    # UGRI COMPLETENESS
    msk4 = ug < 1.5*gr#-0.25
    nugri      = np.sum(msk1&msk2&msk4)
    nugri_spec = np.sum(msk1&msk2&msk3&msk4)
    p = 100.*nugri_spec/nugri
    ugri_compl = '{:02.0f}% ({}/{})'.format(p,nugri_spec,nugri)
    row.append(ugri_compl)
    
    data.append(row)
print tabulate(data,headers, tablefmt="rst")